In [220]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,r2_score,confusion_matrix
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVC
from abc import ABC,abstractmethod
import pandas as pd

In [221]:
#TARGET (interface do domínio específico que o cliente utiliza.)
class RgInterface(ABC):
    @abstractmethod
    def train_model(self):
        pass
    @abstractmethod
    def show_all_shapes(self):
        pass
    @abstractmethod
    def predict(self):
        pass
    @abstractmethod
    def accuracy(self):
        pass

In [222]:
#Interface da classe dos Classificadores
class ClfInterface(ABC):
    @abstractmethod
    def confusion_matrix(self):
        pass

In [223]:

class Regressor(RgInterface):
    def __init__(self,regressor,data,x,y):
        self.__regressor = regressor
        self.__data = data
        self.__x = x
        self.__y = y
        self.__x_train = None
        self.__x_test = None
        self.__y_train = None
        self.__y_test = None
       
    @property
    def train_model(self):
        self.__x_train,self.__x_test,self.__y_train,self.__y_test = train_test_split(self.__x,self.__y,test_size=0.2,shuffle=True,
                                                                                random_state=1)
        self.__regressor.fit(self.__x_train,self.__y_train)
    @property
    def show_all_shapes(self):
        print("x_train shape:",self.__x_train.shape)
        print("x_test shape:",self.__x_test.shape)
        print("y_train shape:",self.__y_train.shape)
        print("y_test shape:",self.__y_test.shape)
    @property
    def predict(self):
        
        
        return self.__regressor.predict(self.__x_test)
    @property
    def accuracy(self):
        previsoes = self.predict
        
        print("Acuracia do modelo: %.2f " % r2_score(previsoes,self.__y_test))
    
    

In [224]:
#ADAPTEE (define uma interface existente que necessita ser adaptada)
class Classifier(ClfInterface):
    def __init__(self,classifier,data,x,y):
        self.classifier = classifier
        self.data = data
        self.x = x
        self.y = y
        self.x_train = None
        self.x_test = None
        self.y_train = None
        self.y_test = None
        
    @property
    def confusion_matrix(self):
        predict = self.classifier.predict(self.x_test)
        print("Matriz confusão: \n",confusion_matrix(predict,self.y_test))
    

In [225]:
#ADAPTER (dapta a interface Adaptee para a interface da classe Target)
class Adapter(RgInterface):
    def __init__(self,classifier):
        self.__classifier = classifier
        
    @property
    def train_model(self):
        self.__classifier.x_train,self.__classifier.x_test,self.__classifier.y_train,self.__classifier.y_test = train_test_split(self.__classifier.x,
                                                                                                                                 self.__classifier.y,
                                                                                                                                 test_size=0.2,
                                                                                                                                 shuffle=True,
                                                                                                                                random_state=1)
        self.__classifier.classifier.fit(self.__classifier.x_train,self.__classifier.y_train)
        
    @property
    def show_all_shapes(self):
        print("x_train shape:",self.__classifier.x_train.shape)
        print("x_test shape:",self.__classifier.x_test.shape)
        print("y_train shape:",self.__classifier.y_train.shape)
        print("y_test shape:",self.__classifier.y_test.shape)
    @property
    def predict(self):
        
        
        return self.__classifier.classifier.predict(self.__classifier.x_test)
    @property
    def accuracy(self):
        previsoes = self.predict
        
        print("Acuracia do modelo: %.2f " % accuracy_score(previsoes,self.__classifier.y_test))
    @property
    def confusion_matrix(self):
        self.__classifier.confusion_matrix

In [226]:
df = pd.read_csv('heart_proba.csv')
y = df.output_proba.values
x = df.drop('output_proba',axis=1).values

In [227]:
reg_line = Regressor(LinearRegression(),df,x,y)

In [228]:
reg_line.train_model

In [229]:
reg_line.show_all_shapes

x_train shape: (241, 14)
x_test shape: (61, 14)
y_train shape: (241,)
y_test shape: (61,)


In [230]:
reg_line.accuracy

Acuracia do modelo: 0.92 


In [231]:
rf = Regressor(RandomForestRegressor(max_depth=15,n_estimators=370),df,x,y)

In [232]:
rf.train_model

In [233]:
rf.accuracy

Acuracia do modelo: 0.91 


In [234]:
data = pd.read_csv('heart.csv')
y = data.output.values
x = data.drop('output',axis=1).values

In [235]:
rf = Classifier(RandomForestClassifier(max_depth=100,n_estimators=10),data,x,y)

In [236]:
AdapterRf = Adapter(rf)

In [237]:
AdapterRf.train_model

In [238]:
AdapterRf.show_all_shapes

x_train shape: (242, 13)
x_test shape: (61, 13)
y_train shape: (242,)
y_test shape: (61,)


In [239]:
AdapterRf.accuracy

Acuracia do modelo: 0.74 


In [240]:
AdapterRf.confusion_matrix

Matriz confusão: 
 [[21  7]
 [ 9 24]]


In [241]:
svm = Classifier(SVC(kernel='linear',C=600),data,x,y)

In [242]:
AdapterSvm = Adapter(svm)

In [243]:
AdapterSvm.train_model

In [244]:
AdapterSvm.show_all_shapes

x_train shape: (242, 13)
x_test shape: (61, 13)
y_train shape: (242,)
y_test shape: (61,)


In [245]:
AdapterSvm.accuracy

Acuracia do modelo: 0.75 


In [246]:
AdapterSvm.train_model

In [247]:
AdapterSvm.confusion_matrix

Matriz confusão: 
 [[19  4]
 [11 27]]
